In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.data import HistoricTradeDataHandler
from EventDriven.event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-04 05:00:31 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [67]:
thetadata_start = '2021-01-01'

In [ ]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
key = 9
with open(f'../input/profitable_weights_{key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{key}.csv').iloc[:, 1:]


In [69]:
trades_ = ttrades__.copy()
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,5.0,504.0,675.0,195.863123,213.759995,89.484359,0.091374,2023-01-04,2023-09-11,250 days,BA
1,8.0,505.0,653.0,171.417877,178.679993,58.096923,0.042365,2023-01-05,2023-08-09,216 days,GLD
2,33.0,507.0,539.0,106.892817,101.029382,-193.493342,-0.054853,2023-01-09,2023-02-24,46 days,SBUX
3,25.0,517.0,535.0,96.125266,89.406462,-167.970102,-0.069896,2023-01-24,2023-02-17,24 days,TSM
4,151.0,518.0,753.0,18.979196,49.243999,4569.985385,1.594630,2023-01-25,2024-01-02,342 days,NVDA
5,2.0,520.0,709.0,148.758846,294.480011,291.442331,0.979580,2023-01-27,2023-10-27,273 days,META
6,3.0,521.0,522.0,62.979658,58.878318,-12.304021,-0.065122,2023-01-30,2023-01-31,1 days,MU
7,3.0,521.0,538.0,359.694542,330.285934,-88.225826,-0.081760,2023-01-30,2023-02-23,24 days,NFLX
8,23.0,527.0,539.0,151.167239,147.029684,-95.163776,-0.027371,2023-02-07,2023-02-24,17 days,AAPL
9,22.0,545.0,708.0,154.328258,170.369995,352.918211,0.103946,2023-03-06,2023-10-26,234 days,AAPL


In [70]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)
pd.Series(weights).sort_values(ascending=False)

TSLA    0.121190
MSFT    0.102841
AAPL    0.077484
SBUX    0.077127
COST    0.067582
NVDA    0.062106
CAT     0.059231
AMD     0.055917
AMZN    0.055496
TSM     0.052926
BAC     0.045990
TLT     0.040779
GOOG    0.038314
GLD     0.033182
NFLX    0.028427
HD      0.027869
BA      0.023677
META    0.009485
MU      0.005000
dtype: float64

In [71]:

from EventDriven.riskmanager import RiskManager, close_cache, spot_cache, chain_cache, oi_cache, LOOKBACKS, order_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.liquidity_threshold = 2
rm.OrderPicker.lookback = 10
rm.OrderPicker.data_availability_threshold = 0.15
date, tick = '2023-07-05', 'AVGO'
date, tick = '2024-08-13', 'TSM'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': .50,
   'dte': 210,
   'moneyness_width': 0.5},
  #  {'direction': 'short',
  # 'rel_strike': .60,
  # 'dte': 270,
  # 'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}



order =  rm.OrderPicker.get_order(tick, date, right, 2, order_settings)
order

{'result': 'SUCCESSFUL',
 'data': {'long': ['TSM20250321C280'],
  'trade_id': '&L:TSM20250321C280',
  'close': 1.515}}

In [72]:
max_cash = {}
cash = 20_000
for s, w in weights.items():
    if w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash

{'TSLA': 2,
 'AMD': 2,
 'MSFT': 2,
 'NVDA': 2,
 'AAPL': 2,
 'SBUX': 2,
 'TSM': 2,
 'COST': 2,
 'AMZN': 2,
 'CAT': 2,
 'BAC': 2,
 'NFLX': 2,
 'GOOG': 2,
 'BA': 2,
 'HD': 2,
 'TLT': 2,
 'GLD': 2,
 'META': 1,
 'MU': 0.5}

In [73]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)

In [74]:
evb_backtest.portfolio.initial_capital

20000

In [75]:
w_map = {x: w  * 0.75 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 100
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .850,
   'dte': 300,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 300,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}

# evb_backtest.portfolio.order_settings = {'type': 'naked',
#  'specifics': [{'direction': 'long',
#    'rel_strike': .750,
#    'dte': 210,
#    'moneyness_width': 0.5},

# ],
#  'name': 'vertical_spread'}


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.executor.commission_rate
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price

{'TSLA': 2,
 'AMD': 2,
 'MSFT': 2,
 'NVDA': 2,
 'AAPL': 2,
 'SBUX': 2,
 'TSM': 2,
 'COST': 2,
 'AMZN': 2,
 'CAT': 2,
 'BAC': 2,
 'NFLX': 2,
 'GOOG': 2,
 'BA': 2,
 'HD': 2,
 'TLT': 2,
 'GLD': 2,
 'META': 1,
 'MU': 0.5}

In [76]:
signals = evb_backtest.bars.signal_df
signals

,Date,BA,GLD,SBUX,TSM,NVDA,META,MU,NFLX,AAPL,AMD,MSFT,GOOG,AMZN,TSLA,COST,CAT,HD,BAC,TLT
0,2023-01-04,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2023-01-05,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2023-01-06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2023-01-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2023-01-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2023-12-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
360,2023-12-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361,2023-12-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
362,2024-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [77]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

36.0

In [78]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  BA  GLD  SBUX  TSM  NVDA  META  MU  NFLX  AAPL  AMD  MSFT  GOOG  \
0 2023-01-04   1    0     0    0     0     0   0     0     0    0     0     0   

   AMZN  TSLA  COST  CAT  HD  BAC  TLT  
0     0     0     0    0   0    0    0  
Processing event: MARKET
Processing event: SIGNAL
Processing event: ORDER
Processing event: FILL
Event queue is empty, processed 4 event(s)
Advancing to next date: 2023-01-05
        Date  BA  GLD  SBUX  TSM  NVDA  META  MU  NFLX  AAPL  AMD  MSFT  GOOG  \
1 2023-01-05   0    1     0    0     0     0   0     0     0    0     0     0   

   AMZN  TSLA  COST  CAT  HD  BAC  TLT  
1     0     0     0    0   0    0    0  
Processing event: MARKET
Processing event: SIGNAL
Not generating order because:TOO_ILLIQUID SignalEvent type:LONG, symbol=GLD, date:2023-01-05 00:00:00, trade_id:GLD20230105C, Order Settings=None
Event queue is empty, processed 2 event(s)
Advancing to next date: 2023-01-06
        Date  BA  GLD  SBUX  TSM  NVDA  META  MU  NFLX  AAPL 

In [79]:
evb_backtest.portfolio.plot_portfolio()

In [80]:
evb_backtest.portfolio.aggregate()

AttributeError: 'DataFrame' object has no attribute 'ReturnPct'

In [48]:
evb_backtest.portfolio.trades

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions,SignalID
0,AAPL,-810.127534,-4.178392,193.884981,6.5,25.424905,962.924905,969.424905,31.924905,159.297370,6.5,-47.013148,802.986852,796.486852,53.513148,5,2022-01-04,2022-01-06,2,&L:AAPL20220916C210&S:AAPL20220916C220,AAPL20220104C
1,MU,-457.904886,-2.269172,201.793861,3.9,16.481583,601.481583,605.381583,20.381583,147.476697,3.9,-26.169908,446.330092,442.430092,30.069908,3,2022-01-04,2022-01-20,16,&L:MU20220916C100&S:MU20220916C105,MU20220104C
2,COST,-62.284835,-0.162419,383.482402,1.3,9.682402,382.182402,383.482402,10.982402,321.197567,1.3,-17.502433,322.497567,321.197567,18.802433,1,2022-04-05,2022-04-29,24,&L:COST20230120C680&S:COST20230120C700,COST20220405C
3,COST,-361.527791,-1.034702,349.402930,2.6,31.205861,696.205861,698.805861,33.805861,337.278069,2.6,-32.843861,677.156139,674.556139,35.443861,2,2022-01-10,2022-01-14,4,&L:COST20230120C560&S:COST20230120C570,COST20220110C
4,COST,-442.400087,-1.232661,358.898373,2.6,35.196746,715.196746,717.796746,37.796746,275.396659,2.6,-41.606682,553.393318,550.793318,44.206682,2,2022-01-18,2022-01-20,2,&L:COST20230120C540&S:COST20230120C550,COST20220118C
5,TSM,-426.870311,-2.204257,193.657194,3.9,29.571582,577.071582,580.971582,33.471582,154.101271,3.9,-21.296187,466.203813,462.303813,25.196187,3,2022-01-18,2022-01-20,2,&L:TSM20220916C140&S:TSM20220916C145,TSM20220118C
6,HD,-441.258227,-2.040509,216.249081,3.9,44.847244,644.847244,648.747244,48.747244,207.489017,3.9,-18.632950,626.367050,622.467050,22.532950,3,2022-01-19,2022-01-24,5,&L:HD20230120C420&S:HD20230120C430,HD20220119C
7,BAC,NaN,NaN,NaN,0.0,0.000000,0.000000,NaN,0.000000,NaN,0.0,-0.000000,0.000000,NaN,0.000000,0,2022-01-19,2022-01-24,5,&L:BAC20221021C42&S:BAC20221021C50,BAC20220119C
8,TSM,-265.206651,-1.258266,210.771599,2.6,18.943197,418.943197,421.543197,21.543197,156.336546,2.6,-24.726908,315.273092,312.673092,27.326908,2,2022-01-21,2022-01-27,6,&L:TSM20230120C150&S:TSM20220916C145,TSM20220121C
9,MSFT,-1098.846926,-5.117004,214.744218,7.8,80.665310,1280.665310,1288.465310,88.465310,189.618384,7.8,-54.489696,1145.510304,1137.710304,62.289696,6,2022-01-21,2022-01-24,3,&L:MSFT20230120C385&S:MSFT20220916C365,MSFT20220121C


In [49]:
evb_backtest.portfolio.get_weighted_holdings()

,AAPL,COST,HD,MU,BAC,TSM,MSFT,GLD,TSLA,UNG,SBUX,CAT,BA,cash,commission,total
datetime,,,,,,,,,,,,,,,,
2022-01-04,1198.980017,1080.288130,671.056371,789.926183,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.000,20000.000000
2022-01-04,1167.055112,1042.498855,671.056371,769.544600,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.130,19909.904237
2022-01-05,1079.555112,1257.498855,671.056371,687.044600,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.130,19954.904237
2022-01-06,1026.041965,1037.498855,671.056371,732.044600,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.195,19726.391089
2022-01-07,1026.041965,1011.499666,671.056371,717.044600,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.221,19685.391901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,694.172412,763.408019,277.655708,431.852010,422.192655,478.697484,926.376614,201.008004,1559.621991,114.221328,586.633211,681.934346,838.505042,9539.247313,1.508,17515.526139
2022-12-27,694.172412,763.408019,277.655708,431.852010,422.192655,478.697484,926.376614,201.008004,1559.621991,114.221328,586.633211,681.934346,838.505042,9539.247313,1.508,17515.526139
2022-12-28,694.172412,763.408019,277.655708,431.852010,422.192655,478.697484,926.376614,201.008004,1559.621991,114.221328,586.633211,654.434346,823.505042,9539.247313,1.508,17473.026139


In [50]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

long                short  \
datetime   symbol                                             
2022-01-04 AAPL     [AAPL20220916C210]   [AAPL20220916C220]   
           COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-05 AAPL     [AAPL20220916C210]   [AAPL20220916C220]   
           COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-06 COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-07 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-10 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-11 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-12 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-13 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-14 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-17 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-18 COST     [COST20230120C540]   [COST20230120C550]   
           MU         [MU20220916C100]     [MU20220916C105]   
           TSM       [TSM20220916C140]    [TSM20220916C145]   
2022-01-19 COST     [COST20230120C540]   [COST20230120C550]   
           HD         [HD20230120C420]     [HD20230120C430]   
           MU         [MU20220916C100]     [MU20220916C105]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
           TSM       [TSM20220916C140]    [TSM20220916C145]   
2022-01-20 HD         [HD20230120C420]     [HD20230120C430]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
2022-01-21 HD         [HD20230120C420]     [HD20230120C430]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
           MSFT     [MSFT20230120C385]   [MSFT20220916C365]   
2022-01-24 TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-25 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-26 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-27 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-01-28 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-01-31 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-01 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-02 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-03 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-04 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-07 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
  

In [373]:


""""
Why do these two return weird stuff after run?
    current_weighted_holdings
    current_positions

I can't reconcile the cost with the data (NVM, haha)

"""
# evb_backtest.portfolio.all_positions

'"\nWhy do these two return weird stuff after run?\n    current_weighted_holdings\n    current_positions\n\nI can\'t reconcile the cost with the data (NVM, haha)\n\n'

In [269]:
order_cache['2024-01-03']

{'BA': {'result': 'TOO_ILLIQUID', 'data': None},
 'BAC': {'result': 'SUCCESSFUL',
  'data': {'long': ['BAC20240719C32'],
   'trade_id': '&L:BAC20240719C32',
   'close': 3.2}},
 'INTC': {'result': 'SUCCESSFUL',
  'data': {'long': ['INTC20240719C39'],
   'trade_id': '&L:INTC20240719C39',
   'close': 12.25}},
 'HD': {'result': 'TOO_ILLIQUID', 'data': None},
 'AMD': {'result': 'SUCCESSFUL',
  'data': {'long': ['AMD20240719C135'],
   'trade_id': '&L:AMD20240719C135',
   'close': 16.625}},
 'QCOM': {'result': 'SUCCESSFUL',
  'data': {'long': ['QCOM20240719C140'],
   'trade_id': '&L:QCOM20240719C140',
   'close': 45.9}},
 'MU': {'result': 'SUCCESSFUL',
  'data': {'long': ['MU20240719C85'],
   'trade_id': '&L:MU20240719C85',
   'close': 29.375}}}

## Extend for  get_port_stats
- Buy & Hold
- Dates
- Trades
- _strategy in Aggregate
- The function

In [214]:
evb_backtest.trades.to_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/output/profitable_trades_options_{key}.csv')